# Exploring Hacker News Post #

- Hacker News is a site started by the startup Y Combinator, where users submits their own stories(known as "posts"), similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.
- The goal of this project to analyze the data to determine the type of posts which rececives more comment and the certain time when created posts gets more comment.
- We're specifically interested in the posts with titles that begin with **Ask HN** or **Show HN**. Users Submit **Ask HN** posts to ask Hacker News community a specific question. Likewise, users submit **Show HN** posts to show the Hacker News community a project, product, or just something interesting.

## Opening and Exploring Data ##

- A [dataset](https://www.kaggle.com/hacker-news/hacker-news-posts) containing the data about the submissions (but note that we have reduced from 300,000 rows to approximatley 20,000 rows by removing all submissions that didn't rececive any comments and then randomly sampling from the remaining submissions.
- Below are the descriptions of the columns:
    - id : the unique identifier from Hacker News for the post
    - title : the title of the posts
    - url : the URL that the posts links to, if the post has a URL
    - num_points : the number of points acquired by the posts , calculated as the total number of upvotes minus the number of downvotes
    - num_comments : the number of comments on the posts
    - author : the username of the person who submitted the post
    - created_at : the date and time of post's submission
- Let's start by opening the dataset and then continue with exploring the data:    

In [1]:
from csv import reader

opened_file = open("/Users/jay/Documents/Guided Project_ Exploring Hacker News Posts/hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [2]:
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## Extracting Ask HN and Show HN Posts ##

- Now that we've removed headers from **hn**, we'll filter our data. Since we're only concerned with the post titles begining with **Ask HN** or **Show HN**, we'll create new lists of lists containing just the data for those titles.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    
    if title.lower().startswith("ask hn"):
        ask_posts.append(title)
    elif title.lower().startswith("show hn"):
        show_posts.append(title)
    else:
        other_posts.append(title)

print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


## Calculating the Average Number of Comments for Ask HN and Show HN Posts ##

In [5]:
total_ask_comments = 0 

for post in hn:
    title = post[1]
    num_comments = int(post[4])
    
    if title in ask_posts:
        total_ask_comments += num_comments
    

avg_ask_comments = total_ask_comments / len(ask_posts)  
print(avg_ask_comments)

14.038417431192661


In [6]:
total_show_comments = 0 

for post in hn:
    title = post[1]
    num_comments = int(post[4])
    
    if title in show_posts:
        total_show_comments += num_comments
        
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

10.31669535283993


On average, ask posts in our sample receive approoximately 14 comments, whereas show posts rececive approximately 10. Since ask posts are more likely to receive comments, we'll focus our reamaining analysis just on this posts.

## Finding the Number of Ask Posts and Comments by Hour Created ##

- Next We'll decide if ask posts created at a certain time are more likely to receive more comments. We'll use the following steps to perform this analysis:
    - Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
    - Calculate the average number of comments ask posts receive by hour created.

In [10]:
import datetime as dt

result_list = []

for post in hn:
    title = post[1]
    created_at = post[6]
    num_comments = int(post[4])
    
    if title in ask_posts:
        result_list.append([created_at, num_comments])

counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    date = row[0]
    comment = row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    
    if time in counts_by_hour:
        counts_by_hour[time] += 1
        comments_by_hour[time] += comment
    else:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comment

comments_by_hour        

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

## Calculating the Average Number of Comments for Ask HN Posts by Hour ##

In [12]:
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr]/counts_by_hour[hr]])

avg_by_hour    

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

## Sorting and Printing Values from a List of Lists ##

In [14]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print(sorted_swap)

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]
[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [

In [19]:
# Sort the values and print the 5 hours with the highest average comments.

print("Top 5 Hours for Ask Posts Comments:")
for avg,hr in sorted_swap[:5]:
    print(
    "{}: {:.2f} average comments per post".format(
    dt.datetime.strptime(hr, "%H").strftime("%H:%M"), avg
    )
    )

Top 5 Hours for Ask Posts Comments:
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that rececives the most comments per post on average is 15:00, with an average of 38.59comments per post. There's about a 60% increase in the number of comments between the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone is Eastern Time in US. According to Indian Standard Time, we could write 15:00 EST as 1:30 a.m. IST. 

## Conclusion ##

In this project, we analyzed ask and show posts to determine which type of post and time rececive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorised as Ask Post and created between 1:30 a.m. to 2:30 a.m IST.

However , it should be noted that the dataset we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that rececived comments, ask posts comments received more comments on average and ask posts created between 1:30 a.m. and 2:30 a.m. IST. 